In [1]:
import pandas as pd
import numpy
import random
import os

from ultralytics import YOLO

import shutil

In [2]:
db1 = pd.read_csv('db1_data.csv')

In [3]:
db1.head()

,filename,xmin,ymin,xmax,ymax,class,width,height
0,db1.jpg,1105,919,1315,1127,object,2272,1704
1,db1.jpg,1005,1271,1166,1363,object,2272,1704
2,db1.jpg,600,1365,745,1494,object,2272,1704
3,db1.jpg,945,1370,1123,1508,object,2272,1704
4,db1.jpg,1536,1432,1898,1522,object,2272,1704


In [4]:
db1['class'].nunique(),db1['width'].nunique(),db1['height'].nunique()

(1, 1, 1)

In [5]:
db1['class_en'] = 0

> Formula for converting pixel annotations to YOLO annotations

``` python
center_x = (x_min + x_max) / (2.0 * image_width)
center_y = (y_min + y_max) / (2.0 * image_height)
width = (x_max - x_min) / image_width
height = (y_max - y_min) / image_height

```

In [6]:
db1.rename(columns={'width':'image_width','height':'image_height'},inplace=True)

In [7]:
# converting to YOLO annotations

db1['center_x'] = (db1['xmin'] + db1['xmax']) / (2*db1['image_width'])
db1['center_y'] = (db1['ymin'] + db1['ymax']) / (2*db1['image_height'])
db1['width'] = (db1['xmax'] - db1['xmin']) / db1['image_width']
db1['height'] = (db1['ymax'] - db1['ymin']) / db1['image_height']

In [14]:
for file in db1['filename'].unique():
    db1[db1['filename']==file][['class_en','center_x','center_y','width','height']].to_csv('./data/labels/train/' + file[:-4] + '.txt', index=False, header=False, sep=' ')

In [15]:
and
image_item_names = [v[:-4] for v in db1['filename'].unique()]
val_items = random.sample(image_item_names,int(.1*len(image_item_names)))
test_items = random.sample(val_items,int(len(val_items)/2))

In [17]:
a
for item in val_items:
    shutil.move('./data/images/train/' + item + '.jpg', './data/images/val/')
    shutil.move('./data/labels/train/' + item + '.txt', './data/labels/val/')

for item in test_items:
    shutil.move('./data/images/val/' + item + '.jpg', './data/images/test/')
    shutil.move('./data/labels/val/' + item + '.txt', './data/labels/test/')

In [21]:
model = YOLO('yolov8n.pt')

In [27]:
results = model.train(data='object_det.yaml', epochs=10, device='mps')

Ultralytics YOLOv8.0.198 🚀 Python-3.8.10 torch-2.1.0 MPS (Apple M1 Pro)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=object_det.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dyn

In [28]:
shutil.copy('./runs/detect/train9/weights/best.pt','./')

'./best.pt'

### prediction using CLI

In [29]:
# predicting from train dataset

!yolo predict model=best.pt source='./data/images/train/db1.jpg'

Ultralytics YOLOv8.0.198 🚀 Python-3.8.10 torch-2.1.0 CPU (Apple M1 Pro)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 /Users/rajatbhardwaj/Downloads/WebMarket_Object_Detection/data/images/train/db1.jpg: 480x640 41 objects, 47.2ms
Speed: 2.2ms preprocess, 47.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict5
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [30]:
# predicting from validation dataset

!yolo predict model=best.pt source='./data/images/val/db20.jpg'

Ultralytics YOLOv8.0.198 🚀 Python-3.8.10 torch-2.1.0 CPU (Apple M1 Pro)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 /Users/rajatbhardwaj/Downloads/WebMarket_Object_Detection/data/images/val/db20.jpg: 480x640 58 objects, 53.1ms
Speed: 2.9ms preprocess, 53.1ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict6
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [31]:
# predicting from test dataset


!yolo predict model=best.pt source='./data/images/test/db132.jpg'

Ultralytics YOLOv8.0.198 🚀 Python-3.8.10 torch-2.1.0 CPU (Apple M1 Pro)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 /Users/rajatbhardwaj/Downloads/WebMarket_Object_Detection/data/images/test/db132.jpg: 480x640 109 objects, 51.1ms
Speed: 2.1ms preprocess, 51.1ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict7
💡 Learn more at https://docs.ultralytics.com/modes/predict


### Prediction using Code

In [32]:
model = YOLO('best.pt')

In [ ]:
'/Users/rajatbhardwaj/Downloads/WebMarket_Object_Detection/data/images/test/db167.jpg'

In [33]:
model.predict('./data/images/test/db210.jpg',save=True)


image 1/1 /Users/rajatbhardwaj/Downloads/WebMarket_Object_Detection/data/images/test/db210.jpg: 480x640 42 objects, 54.8ms
Speed: 2.7ms preprocess, 54.8ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict8


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'object'}
 orig_img: array([[[227, 224, 220],
         [227, 224, 220],
         [229, 226, 222],
         ...,
         [ 14,   8,   9],
         [ 11,   5,   6],
         [  9,   3,   4]],
 
        [[226, 223, 219],
         [227, 224, 220],
         [228, 225, 221],
         ...,
         [ 13,   8,   9],
         [ 11,   6,   7],
         [  9,   4,   5]],
 
        [[224, 223, 219],
         [225, 224, 220],
         [226, 225, 221],
         ...,
         [ 13,   8,   9],
         [ 12,   7,   8],
         [ 10,   5,   6]],
 
        ...,
 
        [[111, 108,  70],
         [143, 142, 104],
         [176, 182, 141],
         ...,
         [226, 227, 225],
         [226, 227, 225],
         [226, 227, 225]],
 
        [[131, 128,  90],
         [149, 148, 110],
         [169, 177, 136],
         ...,
         [227, 228, 226],
   